In [1]:
model_ckp = "xlm-roberta-base"
batch_size = 32

In [2]:
!pip install transformers datasets evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 27.4 MB/s eta 0:00:00


In [3]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 69.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 3.7 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=74ee10bdb874a87acd6d4ad231e529040479f6af1946564eaab8f095e44e6c7e
  Stored in directory: /root/.cache/pip/wheels/4c/8e/7e/72fbc243e1aeecae64a96875432e70d4e92f3d2d18123be004
Successfully built pathtools


In [4]:
import wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [5]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [7]:
from datasets import load_dataset

dataset = load_dataset("minoosh/IEMOCAP_Text")

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Generating Session4 split:   0%|          | 0/1031 [00:00<?, ? examples/s]

Generating Session2 split:   0%|          | 0/1023 [00:00<?, ? examples/s]

Generating Session5 split:   0%|          | 0/1241 [00:00<?, ? examples/s]

Generating Session1 split:   0%|          | 0/1085 [00:00<?, ? examples/s]

Generating Session3 split:   0%|          | 0/1151 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/minoosh___parquet/minoosh--IEMOCAP_Text-8b3962ea9075930e/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/5 [00:00<?, ?it/s]

In [8]:
dataset

DatasetDict({
    Session4: Dataset({
        features: ['emotion', 'transcription'],
        num_rows: 1031
    })
    Session2: Dataset({
        features: ['emotion', 'transcription'],
        num_rows: 1023
    })
    Session5: Dataset({
        features: ['emotion', 'transcription'],
        num_rows: 1241
    })
    Session1: Dataset({
        features: ['emotion', 'transcription'],
        num_rows: 1085
    })
    Session3: Dataset({
        features: ['emotion', 'transcription'],
        num_rows: 1151
    })
})

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_ckp)

In [10]:
def preprocess_function(examples):
    return tokenizer(examples["transcription"], truncation=True)

In [11]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1031 [00:00<?, ? examples/s]

Map:   0%|          | 0/1023 [00:00<?, ? examples/s]

Map:   0%|          | 0/1241 [00:00<?, ? examples/s]

Map:   0%|          | 0/1085 [00:00<?, ? examples/s]

Map:   0%|          | 0/1151 [00:00<?, ? examples/s]

In [12]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [13]:
import evaluate

accuracy = evaluate.load("accuracy")

In [14]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [15]:
dataset = dataset.rename_column("emotion", "label")

In [16]:
labels = dataset["Session1"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

id2label["1"]

'hap'

In [17]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    model_ckp, num_labels=4, id2label=id2label, label2id=label2id
)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

In [18]:
tokenized_dataset = tokenized_dataset.rename_column("emotion", "label")

In [19]:
import torch
train_dataset = torch.utils.data.ConcatDataset([tokenized_dataset['Session1'], tokenized_dataset['Session2'], tokenized_dataset['Session3']])

In [20]:
train_dataset

In [21]:
'''splited_tokenized_dataset = tokenized_dataset['train'].train_test_split(test_size=0.2)
splited_tokenized_dataset'''

"splited_tokenized_dataset = tokenized_dataset['train'].train_test_split(test_size=0.2)\nsplited_tokenized_dataset"

In [22]:
training_args = TrainingArguments(
    output_dir="finetuned_roberta-base-uncased",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    logging_steps=10,
    num_train_epochs=20,
    weight_decay=0.01,
    warmup_ratio=0.1,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
    report_to='wandb',
    run_name='finetune_roberta_base_on_IEMOCAP_text'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=tokenized_dataset["Session4"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Cloning https://huggingface.co/minoosh/finetuned_roberta-base-uncased into local empty directory.


Download file pytorch_model.bin:   0%|          | 15.4k/1.04G [00:00<?, ?B/s]

Download file training_args.bin: 100%|##########| 3.50k/3.50k [00:00<?, ?B/s]

Clean file training_args.bin:  29%|##8       | 1.00k/3.50k [00:00<?, ?B/s]

Download file tokenizer.json:   0%|          | 16.5k/16.3M [00:00<?, ?B/s]

Clean file tokenizer.json:   0%|          | 1.00k/16.3M [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/1.04G [00:00<?, ?B/s]

In [23]:
import torch
torch.cuda.empty_cache()

In [24]:
results = trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3259
  Num Epochs = 20
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 2040
  Number of trainable parameters = 278046724
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: minooshayan97 (minoosh). Use `wandb login --relogin` to force relogin


The following columns in the training set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: transcription. If transcription are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.372000,1.364340,0.337536
2,1.159100,1.198818,0.483026
3,0.962300,1.080171,0.569350
4,0.776600,0.988452,0.623666
5,0.733600,1.039269,0.612027
6,0.628400,1.115044,0.639185
7,0.361600,1.218304,0.640155
8,0.352600,1.236224,0.630456
9,0.315100,1.305780,0.637245
10,0.303500,1.296556,0.634336


The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: transcription. If transcription are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1031
  Batch size = 32
Saving model checkpoint to finetuned_roberta-base-uncased/checkpoint-102
Configuration saved in finetuned_roberta-base-uncased/checkpoint-102/config.json
Model weights saved in finetuned_roberta-base-uncased/checkpoint-102/pytorch_model.bin
tokenizer config file saved in finetuned_roberta-base-uncased/checkpoint-102/tokenizer_config.json
Special tokens file saved in finetuned_roberta-base-uncased/checkpoint-102/special_tokens_map.json
tokenizer config file saved in finetuned_roberta-base-uncased/tokenizer_config.json
Special tokens file saved in finetuned_roberta-base-uncased/special_tokens_map.json
The following columns in the evalu

In [25]:
trainer.evaluate(tokenized_dataset['Session5'])

The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: transcription. If transcription are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1241
  Batch size = 32


{'eval_loss': 1.479914903640747,
 'eval_accuracy': 0.6518936341659952,
 'eval_runtime': 4.3182,
 'eval_samples_per_second': 287.386,
 'eval_steps_per_second': 9.031,
 'epoch': 20.0}

In [26]:
trainer.push_to_hub()

Saving model checkpoint to finetuned_roberta-base-uncased
Configuration saved in finetuned_roberta-base-uncased/config.json
Model weights saved in finetuned_roberta-base-uncased/pytorch_model.bin
tokenizer config file saved in finetuned_roberta-base-uncased/tokenizer_config.json
Special tokens file saved in finetuned_roberta-base-uncased/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 32.0k/1.04G [00:00<?, ?B/s]

remote: Scanning LFS files of refs/heads/main for validity...        
remote: LFS file scan complete.        
To https://huggingface.co/minoosh/finetuned_roberta-base-uncased
   1c3195e..13d14d2  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/minoosh/finetuned_roberta-base-uncased
   1c3195e..13d14d2  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Text Classification', 'type': 'text-classification'}, 'metrics': [{'name': 'Accuracy', 'type': 'accuracy', 'value': 0.6518936341659952}]}
To https://huggingface.co/minoosh/finetuned_roberta-base-uncased
   13d14d2..fa67638  main -> main

   13d14d2..fa67638  main -> main



'https://huggingface.co/minoosh/finetuned_roberta-base-uncased/commit/13d14d2e1a093fca815cbf40352967bfd5b78c8b'

In [27]:
############
#Inference

In [28]:
#text = "This was a masterpiece. Not completely faithful to the books, but enthralling from beginning to end. Might be my favorite of the three."

In [29]:
'''from transformers import pipeline

classifier = pipeline(model="minoosh/my_first_model")
classifier(text)'''

'from transformers import pipeline\n\nclassifier = pipeline(model="minoosh/my_first_model")\nclassifier(text)'